In [1]:
from keras.layers import Input, Dense, Embedding, Convolution1D, MaxPooling1D, MaxPooling2D, Convolution2D, LSTM
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras.regularizers import l2, l1
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import one_hot as oneHOT
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np
df_tweet = pd.read_csv("data/processed_tweet2.csv")
import ast
df_tweet['tweet_pr_st1'] = df_tweet.apply(lambda x: ast.literal_eval(x.tweet_pr_st1),axis =1)
df_tweet['tweet_pr_st2'] = df_tweet.apply(lambda x: ast.literal_eval(x.tweet_pr_st2),axis =1)

df_tweet['subtask_a'] = df_tweet.apply(lambda x: 0 if x.subtask_a=='NOT' else 1,axis = 1)
df_tweet['subtask_b'] = df_tweet.apply(lambda x: 1 if x.subtask_b=='UNT' else 2 if x.subtask_b=='TIN' else 0,axis = 1)
df_tweet['subtask_c'] = df_tweet.apply(lambda x: 2 if x.subtask_c=='IND' else 3 if x.subtask_c=='GRP' else 4 if x.subtask_c =='OTH' else 0,axis = 1)



In [3]:
normalised_corpus = list(df_tweet['tweet_pr_st2'])
labels = list(df_tweet['subtask_a'])

In [4]:
normalised_corpus[:3]

[['ask', 'native', 'american', 'take'],
 ['go',
  'home',
  'drunk',
  'maga',
  'trump',
  'oncoming',
  'fist',
  'united',
  'state',
  'oncoming',
  'fist'],
 ['amazon',
  'investigate',
  'chinese',
  'employee',
  'sell',
  'internal',
  'data',
  'third',
  'party',
  'seller',
  'look',
  'edge',
  'competitive',
  'marketplace',
  'amazon',
  'maga',
  'kag',
  'china',
  'tcot']]

In [5]:
import collections, itertools
word_counter = collections.Counter(list(itertools.chain.from_iterable(normalised_corpus)))

In [6]:
word_ids = list(set(list(itertools.chain.from_iterable(normalised_corpus))))
len(word_ids)

13910

In [7]:
word2idx = {}
word2idx = {token:id+3 for id,token in enumerate(word_ids)}
word2idx['<PAD>'] = 0
word2idx['<START>'] = 1
word2idx['<UNK>'] = 2
word2idx['<UNUSED>'] = 3

# word_index = {k:(v+3) for k,v in word_index.items()}
# word_index["<PAD>"] = 0
# word_index["<START>"] = 1
# word_index["<UNK>"] = 2  
# word_index["<UNUSED>"] = 3

In [8]:
idx2word = {v: k for k,v in word2idx.items()}

In [9]:
sents_as_ids = []
for i in range(len(normalised_corpus)):
    sents_as_ids.append([word2idx[j] for j in normalised_corpus[i]])

In [10]:
normalised_corpus[0:3]

[['ask', 'native', 'american', 'take'],
 ['go',
  'home',
  'drunk',
  'maga',
  'trump',
  'oncoming',
  'fist',
  'united',
  'state',
  'oncoming',
  'fist'],
 ['amazon',
  'investigate',
  'chinese',
  'employee',
  'sell',
  'internal',
  'data',
  'third',
  'party',
  'seller',
  'look',
  'edge',
  'competitive',
  'marketplace',
  'amazon',
  'maga',
  'kag',
  'china',
  'tcot']]

In [11]:
sents_as_ids[0:3]

[[7370, 12279, 7730, 7118],
 [1731, 372, 916, 9323, 121, 8230, 6696, 9589, 13765, 8230, 6696],
 [926,
  7591,
  5693,
  7494,
  7967,
  4571,
  10926,
  2355,
  846,
  1103,
  5762,
  7564,
  2867,
  7142,
  926,
  9323,
  12287,
  12006,
  4312]]

In [12]:
vocab_size = len(word_ids) + 3

In [13]:
# def one_hot(labels):
#     from sklearn.preprocessing import OneHotEncoder
#     encoder = OneHotEncoder()
#     return encoder.fit_transform(np.array(labels).reshape(-1,1)).toarray()

# labels = one_hot(labels)

In [14]:
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  
        wordToIndex = {}  
        indexToWord = {}  

        for line in f:
            record = line.strip().split()
            token = record[0] 
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) 
            
        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  
            wordToIndex[tok] = kerasIdx 
            indexToWord[kerasIdx] = tok 

    return wordToIndex, indexToWord, wordToGlove

In [15]:
def createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, isTrainable):
    vocabLen = len(wordToIndex) + 1  
    embDim = next(iter(wordToGlove.values())).shape[0]  
   
    embeddingMatrix = np.zeros((vocabLen, embDim))  
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word] 

    embeddingLayer = Embedding(vocabLen, embDim, embeddings_initializer=Constant(embeddingMatrix), trainable=isTrainable)
    return embeddingLayer

In [16]:
train_data,test_data,train_labels,test_labels = train_test_split(sents_as_ids,labels,test_size=0.2,stratify=labels)

X_train_enc = pad_sequences(train_data,
                            value=word2idx["<PAD>"],
                            padding='post',
                            maxlen=50)
X_test_enc = pad_sequences(test_data,
                            value=word2idx["<PAD>"],
                            padding='post',
                            maxlen=50)


In [17]:
# put the code here
# !pip install chakin
# import chakin
# chakin.download(number=12, save_dir='embeddings/')

# import os
# os.system("unzip 'embeddings/glove.6B.zip' ")
from tensorflow.contrib.keras.api.keras.initializers import Constant
wordToIndex, indexToWord, wordToGlove = readGloveFile('embeddings/glove.6B.300d.txt')
embeddingLayer = createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, isTrainable=True)
# os.system("rm 'embeddings/glove.6B.zip'")

In [18]:
# from keras.preprocessing.sequence import pad_sequences
# from sklearn.model_selection import train_test_split
# MAXIMUM_LENGTH = 500
# train_data,test_data,train_labels,test_labels = train_test_split(sents_as_ids,labels,test_size=0.2,stratify=labels)

# preprocessed_train_data = pad_sequences(train_data,maxlen=MAXIMUM_LENGTH)
# processed_test_data = pad_sequences(test_data,maxlen=MAXIMUM_LENGTH)

In [19]:
# print('Length of sample train_data before preprocessing:', len(train_data[1]), type(train_data[1]))

In [20]:
# print('Length of sample train_data after preprocessing:', len(preprocessed_train_data[1]), type(train_data[1]))

In [21]:
model = Sequential()
EMBED_SIZE = 100
model.add(embeddingLayer)
# model.add(Embedding(vocab_size,EMBED_SIZE,input_length=MAXIMUM_LENGTH))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(1,activation='sigmoid',input_shape=(1,)))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# put the code here
# model = Sequential()
# model.add(Embedding(vocab_size, 30, input_length=MAXIMUM_LENGTH))
# model.add(Convolution1D(64,5,activation="relu"))
# model.add(Dropout(0.5))
# model.add(Convolution1D(32,3,activation="relu"))
# model.add(Dropout(0.5))
# model.add(Convolution1D(16,3,activation="sigmoid"))
# model.add(MaxPooling1D(5))
# model.add(Flatten())
# model.add(Dense(train_labels.shape[1],activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()



Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         120000300 
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 120,160,801
Trainable params: 120,160,801
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# X_val = np.array(X_train_enc[:10000])
# partial_X_train = np.array(X_train_enc[10000:])

history = model.fit(X_train_enc,
                    train_labels,
                    epochs=15,
                    batch_size=64,
                    validation_split=0.08,
                    verbose=1)

results = model.evaluate(X_test_enc, y_test)
print(results)

Instructions for updating:
Use tf.cast instead.


/home/nv/virtpy3tf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 120000300 elements. This may consume a large amount of memory.
  num_elements)


Train on 9706 samples, validate on 844 samples
Epoch 1/15


ResourceExhaustedError: OOM when allocating tensor with shape[400001,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training/Adam/Variable/Assign (defined at /home/nv/virtpy3tf/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/Variable/Assign', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/nv/virtpy3tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/nv/virtpy3tf/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/nv/virtpy3tf/lib/pytho